### Imports

In [1]:
import pandas as pd
import pathlib
import scipy as sp
import numpy as np
from pathlib import Path

### Classes and functions

In [2]:
class Synonym_Normal_Assumption():

    def __init__(self, df_synonym, score_column="score", se_column="SE", one_sided=True):
        """
        A class to perform a z-test on the scores of non-synonymous variants 
        compared to the distribution of all synonymous variants. This assumes that
        standard errors for each score are known and uses a weighted mean
        (by standard errors) to perform the z-Test.
        """
        self.df_syn = df_synonym
        self.__score = score_column
        self.__se = se_column
        self.__w = "weights"
        self.one_sided = one_sided
        self._fit()
        
    def _fit(self):
        """
        calculate weights and means under the assumption that the synonyms constitute our
        baseline normal distribution
        """
        self.df_syn[self.__w] = 1/self.df_syn[self.__se]**2
        self.weights_sum = self.df_syn[self.__w].sum()
        self.weighted_mean = ((self.df_syn[self.__score]*self.df_syn[self.__w]).sum()) / self.weights_sum
        self.sem = np.sqrt(1/((self.df_syn[self.__se]**2)**-1).sum())


    def get_p_value_two_tailed_standard_normal(self, z_score):
        """calculate the p two-sided"""
        return 2 * (1 - sp.stats.norm.cdf(abs(z_score)))
    
    def get_p_value_one_tailed_standard_normal(self, z_score):
        """calculate the p one-sided"""
        return (1 - sp.stats.norm.cdf(abs(z_score)))

    
    def calc(self):
        """return a calc function that returns the raw p-value"""
        def __calc(row):
            z = self.calculate_z(row)
            if self.one_sided:
                return self.get_p_value_one_tailed_standard_normal(z)
            else:
                return self.get_p_value_two_tailed_standard_normal(z)
        return __calc

    def calculate_z(self, row):
        """Calculate z statistic"""
        z = (row[self.__score] - self.weighted_mean) / self.calculate_sediff(row)
        return z

    def calculate_sediff(self, row):
        """Calculate z statistic"""
        sediff = np.sqrt(self.sem**2 + row[self.__se]**2)
        return sediff

    def benjamini_hochberg_correction(self, p_values, name="adj. p-value(B.H.)"):
        padj = pd.Series(
            sp.stats.false_discovery_control(p_values),
            index=p_values.index,
            name=name
        )
        return padj


### Setttings

In [3]:
out_folder = Path("out")
input_table = out_folder / "Exon5-8_Enrich2_RFS.tsv"
output_file = out_folder / "Exon5-8_Enrich2_RFS_z.tsv"

id_column = "mut_ID"  # id of the variant sequence
score_column="transformed_score_enrich2"  # the score to test on 
se_column="transformed_SE_enrich2"  # # the corresponding standard error
syn_column = "synonym"  # column indicating if this is a synonymous mutation

### Z-Test

In [4]:
# load an input dataframe with scores and standard errors 

df_enrich_rfs = pd.read_csv(input_table, sep="\t")
df_enrich_rfs = df_enrich_rfs.set_index(id_column)
df_enrich_rfs["synonym"] = df_enrich_rfs["type_p"] == "syn"
display(df_enrich_rfs.head())

,hg38_genomic,hg38_cDNA,hg38_protein,read_count_dmso_1,read_count_dmso_2,read_count_dmso_3,read_count_n3a_1,read_count_n3a_2,read_count_n3a_3,read_count_donor,...,Rep2_score,Rep3_SE,Rep3_score,score_CIup,score_CIdown,transformed_score_enrich2,transformed_score_CIup,transformed_score_CIdown,transformed_SE_enrich2,synonym
mut_ID,,,,,,,,,,,,,,,,,,,,,
1,NC_000017.11:g.7675248_7675250del,NM_000546.6:c.376-12_376-10del,NP_000537.3:p.?,842,614,624,8,58,237,3767,...,-2.758029,0.076236,-1.631491,-1.149392,-5.144778,-0.970377,0.298027,-2.238781,0.647145,False
2,NC_000017.11:g.7675247_7675248del,NM_000546.6:c.376-10_376-9del,NP_000537.3:p.?,6506,2489,3260,1103,545,776,25047,...,-1.924381,0.039932,-2.099547,-1.897810,-2.321020,-0.311525,-0.177169,-0.445880,0.068549,False
3,NC_000017.11:g.7675249del,NM_000546.6:c.376-12del,NP_000537.3:p.?,4726,2135,2142,747,164,920,17449,...,-2.969792,0.039410,-1.509519,-1.450645,-3.109950,-0.420024,0.106751,-0.946799,0.268763,False
4,NC_000017.11:g.7675248G>A,NM_000546.6:c.376-12C>T,NP_000537.3:p.?,4441,1576,2528,616,51,489,14982,...,-3.827627,0.049381,-2.306705,-1.940106,-3.798452,-0.793988,-0.204025,-1.383952,0.301002,False
5,NC_000017.11:g.7675248G>C,NM_000546.6:c.376-12C>G,NP_000537.3:p.?,1680,937,622,375,2,113,5006,...,-6.333172,0.102064,-2.366648,-0.863307,-6.155259,-1.200349,0.479672,-2.880369,0.857153,False


In [5]:
# divide the variants into synonymous and non-synonymous mutations
concat = []
df_synonym = df_enrich_rfs.loc[df_enrich_rfs[syn_column], :].copy()
df_non_synonym = df_enrich_rfs.loc[(~df_enrich_rfs[syn_column]), :].copy()

# Perform z-test
syn = Synonym_Normal_Assumption(
        df_synonym, 
        score_column=score_column, 
        se_column=se_column
    )
df_non_synonym["p (one-sided, all)"] = df_non_synonym.apply(syn.calc(), axis=1)

# calcualte Benjamini-Hochberg correction
df_non_synonym["p_adjusted (one-sided, all)"] = syn.benjamini_hochberg_correction(
    df_non_synonym["p (one-sided, all)"]
)

# concat the synonymous and non-synoymous variants dataframes
df_synonym["p (one-sided, all)"] = "NA"
df_synonym["p_adjusted (one-sided, all)"] = "NA"
concat.append(df_synonym)
concat.append(df_non_synonym)
df_enrich_rfs_z = pd.concat(concat)

# reset index and sort
df_enrich_rfs_z = df_enrich_rfs_z.reset_index()
df_enrich_rfs_z = df_enrich_rfs_z.sort_values(id_column)

# display result frame
display(df_enrich_rfs_z.head())

# write to file
df_enrich_rfs_z.to_csv(output_file, index=False, sep="\t")

,mut_ID,hg38_genomic,hg38_cDNA,hg38_protein,read_count_dmso_1,read_count_dmso_2,read_count_dmso_3,read_count_n3a_1,read_count_n3a_2,read_count_n3a_3,...,score_CIup,score_CIdown,transformed_score_enrich2,transformed_score_CIup,transformed_score_CIdown,transformed_SE_enrich2,synonym,weights,"p (one-sided, all)","p_adjusted (one-sided, all)"
377,1,NC_000017.11:g.7675248_7675250del,NM_000546.6:c.376-12_376-10del,NP_000537.3:p.?,842,614,624,8,58,237,...,-1.149392,-5.144778,-0.970377,0.298027,-2.238781,0.647145,False,NaN,0.492765,0.493714
378,2,NC_000017.11:g.7675247_7675248del,NM_000546.6:c.376-10_376-9del,NP_000537.3:p.?,6506,2489,3260,1103,545,776,...,-1.897810,-2.321020,-0.311525,-0.177169,-0.445880,0.068549,False,NaN,0.0,0.0
379,3,NC_000017.11:g.7675249del,NM_000546.6:c.376-12del,NP_000537.3:p.?,4726,2135,2142,747,164,920,...,-1.450645,-3.109950,-0.420024,0.106751,-0.946799,0.268763,False,NaN,0.022544,0.027076
380,4,NC_000017.11:g.7675248G>A,NM_000546.6:c.376-12C>T,NP_000537.3:p.?,4441,1576,2528,616,51,489,...,-1.940106,-3.798452,-0.793988,-0.204025,-1.383952,0.301002,False,NaN,0.292202,0.309629
381,5,NC_000017.11:g.7675248G>C,NM_000546.6:c.376-12C>G,NP_000537.3:p.?,1680,937,622,375,2,113,...,-0.863307,-6.155259,-1.200349,0.479672,-2.880369,0.857153,False,NaN,0.388977,0.400334
